In [1]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine



In [2]:
# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

In [3]:
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)

In [4]:
# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']

In [5]:
oc6_df


,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,...,time_8,time_9,time_10,time_11,time_12,time_13,time_14,streetaddy,movieday,cityaddy
23,2019-06-15,Standard,PG-13,,The Men in Black have always protected the Ear...,Ocala Center 6,Men in Black: International,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Saturday,Ocala
38,2019-06-16,Standard,PG-13,,The Men in Black have always protected the Ear...,Ocala Center 6,Men in Black: International,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
131,2019-06-16,Standard,PG-13,4,The new story follows the heroic efforts of th...,Ocala Center 6,Godzilla: King of the Monsters (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
142,2019-06-16,Standard,PG-13,3.5,This is the story of one of the X-Men's most b...,Ocala Center 6,Dark Phoenix,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
152,2019-06-15,Standard,PG-13,4,The new story follows the heroic efforts of th...,Ocala Center 6,Godzilla: King of the Monsters (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Saturday,Ocala
228,2019-06-14,Standard,PG-13,3.5,This is the story of one of the X-Men's most b...,Ocala Center 6,Dark Phoenix,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala
236,2019-06-16,Standard,PG,4.5,Max the terrier must cope with some major life...,Ocala Center 6,The Secret Life of Pets 2,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
247,2019-06-14,Standard,PG-13,,The Men in Black have always protected the Ear...,Ocala Center 6,Men in Black: International,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala
252,2019-06-15,Standard,PG-13,3.5,This is the story of one of the X-Men's most b...,Ocala Center 6,Dark Phoenix,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Saturday,Ocala
263,2019-06-16,Standard,PG,4.5,Aladdin is a lovable street urchin who meets P...,Ocala Center 6,Aladdin (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala


In [ ]:
"""
This would be an approach using a database
"""
# Put new cinema scrape data into db
#db_directory = os.path.dirname(os.path.abspath(__file__))
#db_file = os.path.join(db_directory, "rinspect.sqlite")
db_file = "cinema_db.sqlite"
movie_table = 'movies' # table for movie data
conn = sqlite3.connect(db_file)
cur = conn.cursor()
df.to_sql('movies', conn, if_exists='replace')

# Build narrative by accessing database


In [ ]:
"""This would be an approach using named tuples"""

In [ ]:
import csv
from collections import namedtuple
from operator import itemgetter

Screening = namedtuple(
    'Screening', '''Location, Theater, Movie, Movie_Rating, Star_Rating, 
    Show_Times, Movie_Format, Synopsis, Date, Searched_City''')

import csv
for show in map(Screening._make, csv.reader(open("scrapes/fandango-2019-06-27.csv", "r"))):
    show.Show_Times = show.Show_Times._replace('p ', ' p.m.')
    if show.Theater == "AMC Lake Square 12" and show.Movie_Format == 'Standard':
        print(show.Movie + show.Date + show.Show_Times)
        

In [ ]:
"""This would be an approach using a dict of dicts.
I'm kinda close on this. Use pandas to organize and concatenate fields.
Concatenate threater + title + format + day into something 
readable. Then move into a dict, directly or via csv."""

In [ ]:
from collections import OrderedDict, defaultdict

df.to_dict(into=OrderedDict)

moviedict = {
    'Ocala6_AladinIMAX_Fri': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sat': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sun': {'times':'3, 2, 3'}
}
fri = str(moviedict['Ocala6_AladinIMAX_Fri']['times'])
sat = str(moviedict['Ocala6_AladinIMAX_Sat']['times'])
sun = str(moviedict['Ocala6_AladinIMAX_Sun']['times'])

if fri == sat and fri == sun: # want all common
    print("Friday-Sunday at " + fri) 
elif fri != sat and fri != sun and sat == sun: # want fri unique
    print("Friday at " + fri + ", Saturday-Sunday at " + sat)
elif sat != fri and sat != sun and fri == sun: # want sat unique
    print("Friday and Sunday at " + fri + ", Saturday at " + sat) 
elif sun != fri and sun != sat and fri == sat: # want sun unique
    print("Friday and Saturday at " + fri + ", Sunday at " + sun)
elif fri != sat and fri !=sun and sat !=sun: # want all unique
    print("Friday at " + fri + ", Saturday at " + sat + ",Sundaay at" + sun)
else:
    print("Show times didn't work out!")

In [ ]:
"""Can I generate the dict up one level, to keep the theater callable
separately?"""


In [ ]:
fri = t1
sat = t2
sun = t3

if t1 in (sat, sun):
    print(q)
if t1 in 

In [ ]:
"""Useful functions, if needed"""

# Convert datetime object into day of the week; But this is accomplished in the df work already.
def get_day(date_object=None):
    if not date_object:
        date_object = datetime.datetime.now()
    stamp = ""
    stamp += datetime.datetime.strftime(date_object, "%A")
    return(stamp)

TypeError: 'Timestamp' object does not support indexing

In [ ]:
for row in oc6_df.itertuples():
    print(row[Index=22])